In [ ]:
import torch
import commons
import utils
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence
import re
from scipy.io.wavfile import write

model_name = 'ljs'
config_file = f"./logs/{model_name}/config.json"
model_file = f"./logs/{model_name}/G_10000.pth"
device = 'cpu' # cuda:0

hps = utils.get_hparams_from_file(config_file)
symbols = list(hps.symbols.pad + hps.symbols.punctuation + hps.symbols.letters + hps.symbols.letters_ipa)

def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).to(device)
_ = net_g.eval()
_ = utils.load_checkpoint(model_file, net_g, None)


text = 'hello my name is andrew'
text = re.sub('[\n]', '', text.strip())
stn_tst = get_text(text, hps)
with torch.no_grad():
    x_tst = stn_tst.to(device).unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1/1)[0][0,0].data.cpu().float().numpy()
write(f'a.wav', hps.data.sampling_rate, audio)
